# Tensorflow tf.data API

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers

In [2]:
data = pd.read_csv("heart.csv")
data.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [3]:
file_data = tf.keras.utils.get_file(origin="https://www.kaggle.com/datasets/fedesoriano/heart-failure-prediction?select=heart.csv")

In [4]:
column_names = ["age", "sex", "cp", "trestbps", "chol", "fbs", "restecg", "thalach", "exang", "oldpeak", "slope", "ca", "thal", "target"]

heart_csv_ds = tf.data.experimental.make_csv_dataset("heart.csv", 
                                                     batch_size=10,
                                                     num_epochs=1,
                                                    column_names=column_names,
                                                    label_name=column_names[-1],
                                                    header=True,
                                                    shuffle=True)

In [5]:
heart_csv_ds

<PrefetchDataset element_spec=(OrderedDict([('age', TensorSpec(shape=(None,), dtype=tf.int32, name=None)), ('sex', TensorSpec(shape=(None,), dtype=tf.int32, name=None)), ('cp', TensorSpec(shape=(None,), dtype=tf.int32, name=None)), ('trestbps', TensorSpec(shape=(None,), dtype=tf.int32, name=None)), ('chol', TensorSpec(shape=(None,), dtype=tf.int32, name=None)), ('fbs', TensorSpec(shape=(None,), dtype=tf.int32, name=None)), ('restecg', TensorSpec(shape=(None,), dtype=tf.int32, name=None)), ('thalach', TensorSpec(shape=(None,), dtype=tf.int32, name=None)), ('exang', TensorSpec(shape=(None,), dtype=tf.int32, name=None)), ('oldpeak', TensorSpec(shape=(None,), dtype=tf.float32, name=None)), ('slope', TensorSpec(shape=(None,), dtype=tf.int32, name=None)), ('ca', TensorSpec(shape=(None,), dtype=tf.int32, name=None)), ('thal', TensorSpec(shape=(None,), dtype=tf.int32, name=None))]), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>

In [6]:
for batch, label in heart_csv_ds.take(1):
    for key, value in batch.items():
        print(f"{key:20s}: {value}")
    print("\n", f"{'label':20s}: {label}")

age                 : [49 61 45 41 47 51 60 64 44 56]
sex                 : [1 1 1 1 1 1 1 1 1 0]
cp                  : [2 3 0 1 0 2 0 2 1 0]
trestbps            : [118 134 142 135 112  94 145 140 120 200]
chol                : [149 234 309 203 204 227 282 335 263 288]
fbs                 : [0 0 0 0 0 0 0 0 0 1]
restecg             : [0 1 0 1 1 1 0 1 1 0]
thalach             : [126 145 147 132 143 154 142 158 173 133]
exang               : [0 0 1 0 0 1 1 0 0 1]
oldpeak             : [0.8 2.6 0.  0.  0.1 0.  2.8 0.  0.  4. ]
slope               : [2 1 1 1 2 2 1 2 2 0]
ca                  : [3 2 3 0 0 1 2 0 0 2]
thal                : [2 2 3 1 2 3 3 2 3 3]

 label               : [0 0 0 1 1 1 0 0 1 0]


In [7]:
%%time
for i, (batch, label) in enumerate(heart_csv_ds.repeat(20)):
    if i % 40 == 0:
        print('.', end='')
print()

................
CPU times: total: 672 ms
Wall time: 437 ms


In [8]:
for element, label in heart_csv_ds.take(1):
    for key, value in element.items():
        print(value)
    print(label)

tf.Tensor([58 64 63 64 44 56 45 59 42 44], shape=(10,), dtype=int32)
tf.Tensor([1 1 0 1 1 0 1 1 0 1], shape=(10,), dtype=int32)
tf.Tensor([0 2 0 2 2 1 0 0 2 2], shape=(10,), dtype=int32)
tf.Tensor([128 140 108 125 140 140 104 110 120 120], shape=(10,), dtype=int32)
tf.Tensor([259 335 269 309 235 294 208 239 209 226], shape=(10,), dtype=int32)
tf.Tensor([0 0 0 0 0 0 0 0 0 0], shape=(10,), dtype=int32)
tf.Tensor([0 1 1 1 0 0 0 0 1 1], shape=(10,), dtype=int32)
tf.Tensor([130 158 169 131 180 153 148 142 173 169], shape=(10,), dtype=int32)
tf.Tensor([1 0 1 1 0 0 1 1 0 0], shape=(10,), dtype=int32)
tf.Tensor([3.  0.  1.8 1.8 0.  1.3 3.  1.2 0.  0. ], shape=(10,), dtype=float32)
tf.Tensor([1 2 1 1 2 1 1 1 1 2], shape=(10,), dtype=int32)
tf.Tensor([2 0 2 0 0 0 0 1 0 0], shape=(10,), dtype=int32)
tf.Tensor([3 2 2 3 2 2 2 3 2 2], shape=(10,), dtype=int32)
tf.Tensor([0 0 0 0 1 1 1 0 1 1], shape=(10,), dtype=int32)


In [9]:
iterator = heart_csv_ds.as_numpy_iterator()
print(next(iterator))

(OrderedDict([('age', array([70, 59, 46, 70, 64, 58, 52, 50, 52, 64])), ('sex', array([1, 1, 1, 1, 0, 0, 1, 1, 1, 1])), ('cp', array([2, 2, 2, 0, 0, 1, 3, 0, 0, 2])), ('trestbps', array([160, 126, 150, 130, 130, 136, 152, 144, 108, 125])), ('chol', array([269, 218, 231, 322, 303, 319, 298, 200, 233, 309])), ('fbs', array([0, 1, 0, 0, 0, 1, 1, 0, 1, 0])), ('restecg', array([1, 1, 1, 0, 1, 0, 1, 0, 1, 1])), ('thalach', array([112, 134, 147, 109, 122, 152, 178, 126, 147, 131])), ('exang', array([1, 0, 0, 0, 0, 0, 0, 1, 0, 1])), ('oldpeak', array([2.9, 2.2, 3.6, 2.4, 2. , 0. , 1.2, 0.9, 0.1, 1.8], dtype=float32)), ('slope', array([1, 1, 1, 1, 1, 2, 1, 1, 2, 1])), ('ca', array([1, 1, 0, 3, 2, 2, 0, 0, 3, 0])), ('thal', array([3, 1, 2, 2, 2, 2, 3, 3, 3, 3]))]), array([0, 0, 0, 0, 1, 0, 1, 0, 1, 0]))


In [12]:
heart_ds = tf.data.Dataset

In [13]:
heart_ds

tensorflow.python.data.ops.dataset_ops.DatasetV2

In [14]:
for element, label in heart_csv_ds.take(1):
    for key, value in element.items():
        heart_ds = heart_ds.from_tensor_slices((value, label))

In [15]:
heart_ds

<TensorSliceDataset element_spec=(TensorSpec(shape=(), dtype=tf.int32, name=None), TensorSpec(shape=(), dtype=tf.int32, name=None))>

In [41]:
# Getting the features
heart_features_dict = {name: np.array(value)
                        for element, label in heart_csv_ds.take(1)
                        for name, value in element.items()}

In [42]:
heart_features_dict

{'age': array([67, 57, 52, 63, 69, 50, 34, 59, 37, 46]),
 'sex': array([0, 1, 1, 1, 0, 1, 1, 0, 0, 1]),
 'cp': array([2, 0, 2, 0, 3, 2, 3, 0, 2, 0]),
 'trestbps': array([115, 132, 172, 130, 140, 129, 118, 174, 120, 120]),
 'chol': array([564, 207, 199, 254, 239, 196, 182, 249, 215, 249]),
 'fbs': array([0, 0, 1, 0, 0, 0, 0, 0, 0, 0]),
 'restecg': array([0, 1, 1, 0, 1, 1, 0, 1, 1, 0]),
 'thalach': array([160, 168, 162, 147, 151, 163, 174, 143, 170, 144]),
 'exang': array([0, 1, 0, 0, 0, 0, 0, 1, 0, 0]),
 'oldpeak': array([1.6, 0. , 0.5, 1.4, 1.8, 0. , 0. , 0. , 0. , 0.8], dtype=float32),
 'slope': array([1, 2, 2, 1, 2, 2, 2, 1, 2, 2]),
 'ca': array([0, 0, 0, 1, 2, 0, 0, 0, 0, 0]),
 'thal': array([3, 3, 3, 3, 2, 2, 2, 2, 2, 3])}

In [45]:
# Getting the labels
heart_features = data.copy()
heart_labels = heart_features.pop("target")
heart_labels

0      1
1      1
2      1
3      1
4      1
      ..
298    0
299    0
300    0
301    0
302    0
Name: target, Length: 303, dtype: int64

In [73]:
heart_features = np.array(heart_features)
heart_features

array([[63.,  1.,  3., ...,  0.,  0.,  1.],
       [37.,  1.,  2., ...,  0.,  0.,  2.],
       [41.,  0.,  1., ...,  2.,  0.,  2.],
       ...,
       [68.,  1.,  0., ...,  1.,  2.,  3.],
       [57.,  1.,  0., ...,  1.,  1.,  3.],
       [57.,  0.,  1., ...,  1.,  1.,  2.]])

In [55]:
heart_features_new = {name: np.array(value) 
                         for name, value in heart_features.items()}

In [49]:
heart_ds = tf.data.Dataset.from_tensor_slices((heart_features_new, heart_labels))
heart_ds

<TensorSliceDataset element_spec=({'age': TensorSpec(shape=(), dtype=tf.int64, name=None), 'sex': TensorSpec(shape=(), dtype=tf.int64, name=None), 'cp': TensorSpec(shape=(), dtype=tf.int64, name=None), 'trestbps': TensorSpec(shape=(), dtype=tf.int64, name=None), 'chol': TensorSpec(shape=(), dtype=tf.int64, name=None), 'fbs': TensorSpec(shape=(), dtype=tf.int64, name=None), 'restecg': TensorSpec(shape=(), dtype=tf.int64, name=None), 'thalach': TensorSpec(shape=(), dtype=tf.int64, name=None), 'exang': TensorSpec(shape=(), dtype=tf.int64, name=None), 'oldpeak': TensorSpec(shape=(), dtype=tf.float64, name=None), 'slope': TensorSpec(shape=(), dtype=tf.int64, name=None), 'ca': TensorSpec(shape=(), dtype=tf.int64, name=None), 'thal': TensorSpec(shape=(), dtype=tf.int64, name=None)}, TensorSpec(shape=(), dtype=tf.int64, name=None))>

In [53]:
heart_batches = heart_ds.shuffle(len(heart_labels)).batch(32)
heart_batches

<BatchDataset element_spec=({'age': TensorSpec(shape=(None,), dtype=tf.int64, name=None), 'sex': TensorSpec(shape=(None,), dtype=tf.int64, name=None), 'cp': TensorSpec(shape=(None,), dtype=tf.int64, name=None), 'trestbps': TensorSpec(shape=(None,), dtype=tf.int64, name=None), 'chol': TensorSpec(shape=(None,), dtype=tf.int64, name=None), 'fbs': TensorSpec(shape=(None,), dtype=tf.int64, name=None), 'restecg': TensorSpec(shape=(None,), dtype=tf.int64, name=None), 'thalach': TensorSpec(shape=(None,), dtype=tf.int64, name=None), 'exang': TensorSpec(shape=(None,), dtype=tf.int64, name=None), 'oldpeak': TensorSpec(shape=(None,), dtype=tf.float64, name=None), 'slope': TensorSpec(shape=(None,), dtype=tf.int64, name=None), 'ca': TensorSpec(shape=(None,), dtype=tf.int64, name=None), 'thal': TensorSpec(shape=(None,), dtype=tf.int64, name=None)}, TensorSpec(shape=(None,), dtype=tf.int64, name=None))>

In [39]:
for element in heart_ds.as_numpy_iterator():
    print(element)

{'age': 59, 'sex': 1, 'cp': 3, 'trestbps': 134, 'chol': 204, 'fbs': 0, 'restecg': 1, 'thalach': 162, 'exang': 0, 'oldpeak': 0.8, 'slope': 2, 'ca': 2, 'thal': 2}
{'age': 61, 'sex': 1, 'cp': 0, 'trestbps': 148, 'chol': 203, 'fbs': 0, 'restecg': 1, 'thalach': 161, 'exang': 0, 'oldpeak': 0.0, 'slope': 2, 'ca': 1, 'thal': 3}
{'age': 49, 'sex': 0, 'cp': 0, 'trestbps': 130, 'chol': 269, 'fbs': 0, 'restecg': 1, 'thalach': 163, 'exang': 0, 'oldpeak': 0.0, 'slope': 2, 'ca': 0, 'thal': 2}
{'age': 48, 'sex': 1, 'cp': 0, 'trestbps': 124, 'chol': 274, 'fbs': 0, 'restecg': 0, 'thalach': 166, 'exang': 0, 'oldpeak': 0.5, 'slope': 1, 'ca': 0, 'thal': 3}
{'age': 42, 'sex': 1, 'cp': 0, 'trestbps': 136, 'chol': 315, 'fbs': 0, 'restecg': 1, 'thalach': 125, 'exang': 1, 'oldpeak': 1.8, 'slope': 1, 'ca': 0, 'thal': 1}
{'age': 70, 'sex': 1, 'cp': 0, 'trestbps': 130, 'chol': 322, 'fbs': 0, 'restecg': 0, 'thalach': 109, 'exang': 0, 'oldpeak': 2.4, 'slope': 1, 'ca': 3, 'thal': 2}
{'age': 52, 'sex': 1, 'cp': 1, 'tr

In [56]:
inputs = {}

for name, column in heart_features.items():
    dtype = column.dtype
    if dtype == object:
        dtype = tf.string
    else:
        dtype = tf.float32

    inputs[name] = tf.keras.Input(shape=(1,), name=name, dtype=dtype)

inputs

{'age': <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'age')>,
 'sex': <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'sex')>,
 'cp': <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'cp')>,
 'trestbps': <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'trestbps')>,
 'chol': <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'chol')>,
 'fbs': <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'fbs')>,
 'restecg': <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'restecg')>,
 'thalach': <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'thalach')>,
 'exang': <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'exang')>,
 'oldpeak': <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'oldpeak')>,
 'slope': <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'slope')>,
 'ca': <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'ca')>,
 'thal': <Ke

In [58]:
numeric_inputs = {name:input for name,input in inputs.items()
                  if input.dtype==tf.float32}

x = layers.Concatenate()(list(numeric_inputs.values()))
norm = layers.Normalization()
norm.adapt(np.array(data[numeric_inputs.keys()]))
all_numeric_inputs = norm(x)

all_numeric_inputs

<KerasTensor: shape=(None, 13) dtype=float32 (created by layer 'normalization_1')>

In [59]:
preprocessed_inputs = [all_numeric_inputs]

In [62]:
for name, input in inputs.items():
    if input.dtype == tf.float32:
        continue

    lookup = layers.StringLookup(vocabulary=np.unique(titanic_features[name]))
    one_hot = layers.CategoryEncoding(num_tokens=lookup.vocabulary_size())

    x = lookup(input)
    x = one_hot(x)
    preprocessed_inputs.append(x)

In [63]:
preprocessed_inputs_cat = layers.Concatenate()(preprocessed_inputs)

heart_preprocessing = tf.keras.Model(inputs, preprocessed_inputs_cat)

In [69]:
def heart_model(preprocessing_head, inputs):
    body = tf.keras.Sequential([
    layers.Dense(64),
    layers.Dense(1)
  ])

    preprocessed_inputs = preprocessing_head(inputs)
    result = body(preprocessed_inputs)
    model = tf.keras.Model(inputs, result)
    model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                optimizer=tf.keras.optimizers.Adam())
    return model
heart_model = heart_model(heart_preprocessing, inputs)

In [70]:
heart_model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 age (InputLayer)               [(None, 1)]          0           []                               
                                                                                                  
 ca (InputLayer)                [(None, 1)]          0           []                               
                                                                                                  
 chol (InputLayer)              [(None, 1)]          0           []                               
                                                                                                  
 cp (InputLayer)                [(None, 1)]          0           []                               
                                                                                            

In [74]:
EPOCHS = 5   
model.fit(x=heart_features, y=heart_labels, epochs=EPOCHS)

Epoch 1/5


ValueError: in user code:

    File "C:\Users\pedro\Proyectos\myenv\lib\site-packages\keras\engine\training.py", line 1249, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\pedro\Proyectos\myenv\lib\site-packages\keras\engine\training.py", line 1233, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\pedro\Proyectos\myenv\lib\site-packages\keras\engine\training.py", line 1222, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\pedro\Proyectos\myenv\lib\site-packages\keras\engine\training.py", line 1023, in train_step
        y_pred = self(x, training=True)
    File "C:\Users\pedro\Proyectos\myenv\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "C:\Users\pedro\Proyectos\myenv\lib\site-packages\keras\engine\input_spec.py", line 277, in assert_input_compatibility
        raise V

In [81]:
model = tf.keras.Sequential([
  layers.Dense(64),
  layers.Dense(1)
])

model.compile(loss = tf.keras.losses.MeanSquaredError(),
                      optimizer = tf.keras.optimizers.Adam())

In [83]:
model.fit(heart_features, heart_labels, epochs=10)

Epoch 1/10
10/10 [==============================] - 0s 2ms/step - loss: 0.1344
Epoch 2/10
10/10 [==============================] - 0s 1ms/step - loss: 0.1460
Epoch 3/10
10/10 [==============================] - 0s 1ms/step - loss: 0.1342
Epoch 4/10
10/10 [==============================] - 0s 1ms/step - loss: 0.1401
Epoch 5/10
10/10 [==============================] - 0s 2ms/step - loss: 0.1389
Epoch 6/10
10/10 [==============================] - 0s 1ms/step - loss: 0.1404
Epoch 7/10
10/10 [==============================] - 0s 2ms/step - loss: 0.1372
Epoch 8/10
10/10 [==============================] - 0s 2ms/step - loss: 0.1392
Epoch 9/10
10/10 [==============================] - 0s 2ms/step - loss: 0.1463
Epoch 10/10
10/10 [==============================] - 0s 2ms/step - loss: 0.1532
